In [1]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import openpyxl

In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart

In [3]:
chromedriver_path= '.\chromedriver.exe'

In [4]:
driver = webdriver.Chrome(executable_path=chromedriver_path)

In [5]:
# yatra='https://flight.yatra.com/air-search-ui/dom2/trigger?type=R&viewName=normal&flexi=0&noOfSegments=2&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date=30%2F09%2F2022&arrivalDate=04%2F10%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=456183416202'
# cleartrip='https://www.cleartrip.com/flights/results?adults=3&childs=0&infants=0&class=Economy&depart_date=14/11/2022&return_date=18/11/2022&from=DEL&to=IXZ&intl=n&origin=DEL%20-%20New%20Delhi,%20IN&destination=IXZ%20-%20Port%20Blair,%20IN&sd=1660963642740&rnd_one=R&sourceCountry=New%20Delhi&destinationCountry=Port%20Blair'
# kayak='https://www.kayak.co.in/flights/DEL-BOM/2022-09-21-flexible/2022-09-27-flexible?sort=bestflight_a'
# driver.get(kayak)

In [6]:
pop_close='//*[@class="B5lA-button"]'
closetry='//*[@class="bBPb-closeIcon"]'
#driver.find_element_by_xpath(pop_close).click()
#driver.find_element_by_xpath(closetry).click()

In [7]:
def load_more():
    try:
        more_results='//*[@class="moreButton"]'
        driver.find_element_by_xpath(more_results).click()
        print('sleeping....')
        sleep(randint(5,10))
    except:
        pass

In [8]:
def page_scrape():
    """This function takes care of the scraping part"""
    
    xp_sections = '//*[@class="section duration allow-multi-modal-icons"]'
    sections = driver.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2] # This is to separate the two flights
    section_b_list = sections_list[1::2] # This is to separate the two flights
    
    # if you run into a reCaptcha, you might want to do something about it
    # you will know there's a problem if the lists above are empty
    # this if statement lets you exit the bot or do something else
    # you can add a sleep here, to let you solve the captcha and continue scraping
    # i'm using a SystemExit because i want to test everything from the start
 
    # I'll use the letter A for the outbound flight and B for the inbound
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        # Separate the time from the cities
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        # Separate the time from the cities
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))
 
    xp_dates = '//div[@class="section stacked-carriers with-date"]'
    dates = driver.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_date_list = dates_list[::2]
    b_date_list = dates_list[1::2]
    
    #**************************
    # Separating the weekday from the day
#     a_day = [value.split()[0] for value in a_date_list]
#     a_weekday = [value.split()[1] for value in a_date_list]
#     b_day = [value.split()[0] for value in b_date_list]
#     b_weekday = [value.split()[1] for value in b_date_list]
    
#     print(a_day)
    
    
    #*********************************
    
    # getting the prices
    
    xp_prices = '//*[@class="price option-text"]'
    prices = driver.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('₹ ','').replace(',','') for price in prices if price.text != '']
    prices_list = list(map(int, prices_list))
    
    #solving problem of prices 
    pl=[]
    for i in range(len(prices_list)):
        if i<len(a_duration):
            pl.append(prices_list[i])

    # the stops are a big list with one leg on the even index and second leg on odd index
#     xp_stops = '//*[@class="section stops"]'
#     stops = driver.find_elements_by_xpath(xp_stops)
#     stops_list = [stop.text[0].replace('n','0') for stop in stops]
#     a_stop_list = stops_list[::2]
#     b_stop_list = stops_list[1::2]

    xp_stops_cities = '//div[@class="section stops"]/div[2]'
    stops_cities = driver.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]

    # this part gets me the airline company and the departure and arrival times, for both legs
    xp_schedule = '//div[@class="section times"]'
    schedules = driver.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
#         print(hours_list)
#         carrier_list.append(schedule.text.split('\n')[1])
    # split the hours and carriers, between a and b legs
    a_hours = hours_list[::2]
#     a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
#     b_carrier = carrier_list[1::2]
#    print(len(prices_list))
    cols = ([ 'Out Time', 'Out Cities', 'Out Duration', 'Out Stop Cities','Return Time', 'Return Cities', 'Return Duration', 'Return Stop Cities', 'Price'])
#'Out Day','Out Weekday','Return Day', , 'Return Weekday', 'Out Stops', 'Return Stops', 'Return Airline', 'Out Airline'
    flights_df = pd.DataFrame({#'Out Day': a_day,
                               #'Out Weekday': a_weekday,
                               'Out Time': a_hours,
                               'Out Cities': a_section_names,
                               'Out Duration': a_duration,
#                                'Return Day': b_day,
#                                'Return Weekday': b_weekday,
                               'Return Stop Cities': b_stop_name_list,
#                                'Out Airline': a_carrier,
                                'Return Time': b_hours,
                                'Return Duration': b_duration,
                                'Return Cities': b_section_names,
#                                'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
#                                'Return Stops': b_stop_list,
#                                'Return Airline': b_carrier,                           
                               'Price': pl})[cols]
    
    flights_df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped
    return flights_df

In [14]:
def start_yatra(from_city, to_city, date_start, date_end):
    kayak= ('https://www.kayak.co.in/flights/'+ from_city + '-' + to_city +'/'+ date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)
    sleep(randint(8,10))
    try:
        pop_close='//a[@class="B5lA-button"]'
        driver.find_element_by_xpath(pop_close).click()
    except Exception as e:
        pass
    sleep(randint(6,9))
    print('loading more...')
    load_more()
    
    #best
    
#     print('scrape best')
#     df_best=page_scrape()
#     df_best['sort']='best'
#     sleep(randint(60,80))
#     print("best scrapped, cheap scrape")
    
    #lowest price
    #matrix= driver.find_element_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
    pmat='//*[@class="jPY1-inner jPY1-mod-inner-default"]'
    matrix= driver.find_elements_by_xpath(pmat)
    matrix_price=[price.text.replace('₹ ','').replace(',','') for price in matrix]
    matrix_price=list(map(int,matrix_price))
    matrix_min=min(matrix_price)
    matrix_avg= sum(matrix_price)/len(matrix_price)
    
    print('cheapest results')
#     #cheap='//a[@data-code="price"]'
    #heap='//*[@class="js-label _itL _fI _igh _gB _iai _kK7"]'
    cheap='//*[@id="lL5p-price_aTab"]/div[1]/div/div'
    c=driver.find_elements_by_xpath(cheap)
    for s in c:
        if s.text=='Cheapest':
            s.click()
            print(quick)
    sleep(randint(60,80))
    print('starting scrape 2')
    df_cheap=page_scrape()
    df_cheap['sort']='cheap'
    sleep(randint(60,80))
    
#     #fastest
    
#     print('fastest results')
#     #fast='//a[@data-code="duration"]'
#     #//*[@id="k5Ti-price_aTab"]/div[1]/div/div
#     fast='//*[@class="js-label _itL _fI _igh _gB _iai _kK7"]'
#     f=driver.find_elements_by_xpath(fast)
#     for s in f:
#         if s.text=='Quickest':
#             driver.find_elements_by_xpath(fast)[0].click()
#             print(fast)
#     sleep(randint(60,80))
#     print('starting scrape 3')
#     df_fast=page_scrape()
#     df_fast['sort']='fast'
#     sleep(randint(60,80))
    
#     df= df_cheap.append(df_best).append(df_fast)
    df=df_best
    df.to_excel('./flights_{}-{}_from_{}_to_{}.xlsx'.format(from_city, to_city, date_start, date_end), index=False)
    print('saved')
    

In [15]:
# from_city = input('From which city? ')
# to_city = input('Where to? ')
# date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
# date_end = input('Return when? Please use YYYY-MM-DD format only ')
from_city='DEL'    
to_city='BOM'
date_start='2022-09-21'
date_end='2022-09-27'
start_yatra(from_city , to_city , date_start , date_end) 
#for n in range(0,5):
#print('iteration was complete')
    
    # Wait 4 hours
    #sleep(60*60*4)
print('sleep finished.....')

loading more...
cheapest results
starting scrape 2


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=104.0.5112.102)
